In [1]:
import os
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit, lag
from pyspark.sql.window import Window
from pyspark.sql.types import LongType 
import pyspark.sql.types as T

In [2]:
# !!!! check that the timezone is correct for cuebiq data before running anything. See http://wrschneider.github.io/2019/09/01/timezones-parquet-redshift.html
assert spark.conf.get("spark.sql.session.timeZone") == 'Etc/UTC'

In [3]:
results_path = '/dbfs/mnt/Geospatial/results/'
start_date='2020-01-01'
end_date='2020-05-01'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
start_hour_day = 8
end_hour_day = 20


# Compute share of activity in unique neighborhoods

In [6]:
pings_geocoded = spark.read.parquet(os.path.join(results_path.replace("/dbfs/", ""),source, 'processed',country,'pings_geocoded/*.parquet')).withColumn("date",  F.date_format('time','yyyy-MM-dd')).withColumn("hour",  F.hour('time'))
pings_geocoded = pings_geocoded.where((pings_geocoded.time >= start_date) & (pings_geocoded.time <= end_date))


In [7]:
n_admin_date_id=pings_geocoded.groupby(['date','cuebiq_id']).agg(F.countDistinct(admin_id).alias("nunique"), F.count(lit(1)).alias("n_pings"))

day_df = pings_geocoded.filter((col("hour") >= start_hour_day) & (col("hour") < end_hour_day))
n_admin_daytime_id = day_df.groupby(['date','cuebiq_id']).agg(F.countDistinct(admin_id).alias("nunique"), F.count(lit(1)).alias("n_pings"))

night_df  = pings_geocoded.filter(((col("hour") >= end_hour_day) & (col("hour") <= 23)) |((col("hour") >= 0) & (col("hour") < start_hour_day)))
n_admin_nighttime_id = night_df.groupby(['date','cuebiq_id']).agg(F.countDistinct(admin_id).alias("nunique"), F.count(lit(1)).alias("n_pings"))


In [8]:
n_admin_date_id = n_admin_date_id.withColumn('pct_unique', col('nunique') / col('n_pings'))
n_admin_nighttime_id = n_admin_nighttime_id.withColumn('pct_unique', col('nunique') / col('n_pings'))
n_admin_daytime_id = n_admin_daytime_id.withColumn('pct_unique', col('nunique') / col('n_pings'))

In [9]:

n_admin_date_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'n_admin_date_id.csv'), index=False)
n_admin_daytime_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'n_admin_daytime_id.csv'), index = False)
n_admin_nighttime_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'n_admin_nighttime_id.csv'), index = False)


In [10]:
dbfs

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3055583274555387> in <module> 
 ----> 1 dbfs

 NameError : name 'dbfs' is not defined

# Figures